In [1]:
if "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import datetime
import gc
import pandas as pd
import joypy

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.datasets as datasets

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import timedelta

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/dist

In [3]:
# PCA_ = PCA(n_components=60).fit(X_norm)
# X_PCA = PCA_.transform(X_norm)
# # X_TSNE = TSNE(n_components=60, learning_rate="auto", init='random', random_state=0).fit_transform(X_PCA)
# reducer = umap.UMAP(min_dist=0.9, n_components=60)
# X_UMAP = reducer.fit_transform(X_norm)
# X_batch = np.reshape(X_UMAP, (X_UMAP.shape[0]//5,5,X_UMAP.shape[1]))
# # X_UMAP_Norm = Normalizer().fit_transform(X_UMAP)

In [4]:
root_audios_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_audios.csv"
root_recorders_Zamuro = f"{root}/Zamuro/Complementary_Files/zamuro_recorders.csv"

audios = pd.read_csv(root_audios_Zamuro, index_col=0)
recorders = pd.read_csv(root_recorders_Zamuro, index_col=0)

def combinar_nombre_ubicacion(row):
    return f"{row['field_number_PR']}_{row['Filename']}"

# Aplicando la función a cada fila del DataFrame para crear la nueva columna
audios['Filename_'] = audios.apply(combinar_nombre_ubicacion, axis=1)

audios.set_index("Filename_", inplace=True)

# AE Model

In [39]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

models = [1,3,5,7,10,20,30,40,50]

accuracies = []
f1_scores = []
recalls = []
for  i, mod_id in enumerate(models):

    X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/epochs_50/{mod_id}/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
    X_ae = np.asarray(X_ae)
    y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/epochs_50/{mod_id}/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))
    
    y = np.asarray(y_path)
    y_2 = y[:,0]
    for i in range(len(y_2)):
        y_2[i] = y_2[i][0:-2] 
    y_2 = list(y_2)
    y_3 = np.repeat(y_2, 5)
    labels_ae = []
    for i in range(len(y_3)):
        labels_ae.append(audios.loc[y_3[i], "cover"])
        
    X_train, X_test, y_train, y_test = train_test_split(X_ae, labels_ae, test_size=0.2,random_state=0)
    
    clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
    clf_rf.fit(X_train, y_train)

    y_pred_rf = clf_rf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred_rf)
    f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
    recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
    print("Accuracy:", accuracy)
    print("f1:", f1_score)
    print("recall", recall)
    
    accuracies.append(accuracy)
    f1_scores.append(f1_score)
    recalls.append(recall)
    
np.save(f"{root}/Zamuro/temporal_zamuro/accuracies.npy", accuracies)
np.save(f"{root}/Zamuro/temporal_zamuro/f1_scores.npy", f1_scores)
np.save(f"{root}/Zamuro/temporal_zamuro/recalls.npy", recalls)

# X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
# X_ae = np.asarray(X_ae)
# y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))

# scaler = StandardScaler().fit(X_ae)
# X_scaled = scaler.transform(X_ae)

# Normalizer_ = Normalizer().fit(X_ae)
# X_norm = Normalizer_.transform(X_ae)


FileNotFoundError: [Errno 2] No such file or directory: '/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics/Zamuro/temporal_zamuro/Features/epochs_50/1/AE_features_Zamuro.pth'

In [16]:
X_ae = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
X_ae = np.asarray(X_ae)
y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/log_standarization_epochs_10/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))

y = np.asarray(y_path)
y_2 = y[:,0]
for i in range(len(y_2)):
    y_2[i] = y_2[i][0:-2] 
y_2 = list(y_2)
y_3 = np.repeat(y_2, 5)
labels_ae = []
for i in range(len(y_3)):
    labels_ae.append(audios.loc[y_3[i], "cover"])
    
X_ae_unflat = np.reshape(X_ae, [X_ae.shape[0]//5,5,X_ae.shape[1]])
df_ae = pd.DataFrame(X_ae)
df_ae["y"] = y_3

In [18]:
def extract_parts(row):
    parts = row.split('_')
    location = parts[0]
    date = parts[1]
    time = parts[2].split('.')[0]  # Eliminar la extensión .WAV
    day = date[-2:]  # Últimos dos caracteres para el día
    hour = time[:2]
    return pd.Series([location, day, hour])

# Aplicar la función a la columna 'y' y crear nuevas columnas
df_ae[['location', 'day', 'hour']] =df_ae['y'].apply(extract_parts)

def define_hour_stage(hour):
    hour = int(hour)
    if 5 <= hour <= 8:
        return 'morning'
    elif 9 <= hour <= 16:
        return 'day'
    else:
        return 'night'

df_ae['hour_stage'] =df_ae['hour'].apply(define_hour_stage)
df_ae.set_index("y", inplace=True, drop=False)
df_ae['cover'] = df_ae.index.map(audios['cover'])

In [20]:
df_ae.head()

,0,1,2,3,4,5,6,7,8,9,...,5180,5181,5182,5183,y,location,day,hour,hour_stage,cover
y,,,,,,,,,,,,,,,,,,,,,
RZUH02_20220906_043000.WAV,10.159684,9.957931,9.970580,10.340969,9.902819,10.074280,10.020636,10.126237,9.856021,10.513670,...,9.258468,9.063374,8.588468,9.649255,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,10.014838,10.013358,10.164495,10.009226,10.024938,9.821374,9.865287,10.252872,10.066131,10.501209,...,8.508420,8.417641,7.922295,8.536418,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,9.874201,9.613174,9.428842,9.855727,9.184982,9.542309,9.898331,10.138428,10.234679,10.439282,...,7.548015,8.238778,8.112399,8.383661,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,10.383832,10.089753,10.019128,10.162333,10.204617,10.145849,10.132951,9.966949,9.962975,10.546733,...,8.496480,7.431211,8.005023,8.612472,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,10.106177,10.247113,10.203444,10.545758,10.467957,10.384063,10.207061,10.170845,10.103601,10.529855,...,8.034658,8.641006,8.365255,9.214890,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture


In [22]:
df_day.head()

,0,1,2,3,4,5,6,7,8,9,...,5180,5181,5182,5183,y,location,day,hour,hour_stage,cover
y,,,,,,,,,,,,,,,,,,,,,
RZUH02_20220903_090000.WAV,9.426662,10.290031,10.414343,10.073651,8.735041,8.646876,9.901097,11.451177,9.508819,10.445171,...,8.725511,8.730370,8.713900,8.715708,RZUH02_20220903_090000.WAV,RZUH02,03,09,day,pasture
RZUH02_20220903_090000.WAV,10.561423,8.856417,12.907985,9.440134,9.778910,9.067059,10.927027,12.237653,6.114228,10.550804,...,8.703635,8.685799,8.712638,8.720999,RZUH02_20220903_090000.WAV,RZUH02,03,09,day,pasture
RZUH02_20220903_090000.WAV,8.674242,9.537881,7.879604,8.087102,8.667194,11.172038,8.910363,9.144228,5.184620,10.397343,...,8.718995,8.699444,8.683823,8.705215,RZUH02_20220903_090000.WAV,RZUH02,03,09,day,pasture
RZUH02_20220903_090000.WAV,8.565351,9.218843,11.452531,6.151741,10.257081,10.478706,10.596461,11.070013,9.611480,10.162099,...,8.690443,8.728479,8.647233,8.672675,RZUH02_20220903_090000.WAV,RZUH02,03,09,day,pasture
RZUH02_20220903_090000.WAV,8.282427,11.023909,8.498660,11.088573,7.333527,11.891114,10.873700,10.641357,12.115484,9.876040,...,8.660295,8.710748,8.684983,8.699404,RZUH02_20220903_090000.WAV,RZUH02,03,09,day,pasture


In [23]:
df_day = df_ae[df_ae['day'].isin(["03"])]
# df_day.reset_index(inplace=True)
print(df_day.head())
y = df_day.loc[:,"cover"]
y

                                    0          1          2          3  \
y                                                                        
RZUH02_20220903_090000.WAV   9.426662  10.290031  10.414343  10.073651   
RZUH02_20220903_090000.WAV  10.561423   8.856417  12.907985   9.440134   
RZUH02_20220903_090000.WAV   8.674242   9.537881   7.879604   8.087102   
RZUH02_20220903_090000.WAV   8.565351   9.218843  11.452531   6.151741   
RZUH02_20220903_090000.WAV   8.282427  11.023909   8.498660  11.088573   

                                    4          5          6          7  \
y                                                                        
RZUH02_20220903_090000.WAV   8.735041   8.646876   9.901097  11.451177   
RZUH02_20220903_090000.WAV   9.778910   9.067059  10.927027  12.237653   
RZUH02_20220903_090000.WAV   8.667194  11.172038   8.910363   9.144228   
RZUH02_20220903_090000.WAV  10.257081  10.478706  10.596461  11.070013   
RZUH02_20220903_090000.WAV   7.333527

y
RZUH02_20220903_090000.WAV    pasture
RZUH02_20220903_090000.WAV    pasture
RZUH02_20220903_090000.WAV    pasture
RZUH02_20220903_090000.WAV    pasture
RZUH02_20220903_090000.WAV    pasture
                               ...   
RZUD03_20220903_211500.WAV    savanna
RZUD03_20220903_211500.WAV    savanna
RZUD03_20220903_211500.WAV    savanna
RZUD03_20220903_211500.WAV    savanna
RZUD03_20220903_211500.WAV    savanna
Name: cover, Length: 5765, dtype: object

In [38]:
df_ae

,0,1,2,3,4,5,6,7,8,9,...,5180,5181,5182,5183,y,location,day,hour,hour_stage,cover
y,,,,,,,,,,,,,,,,,,,,,
RZUH02_20220906_043000.WAV,10.159684,9.957931,9.970580,10.340969,9.902819,10.074280,10.020636,10.126237,9.856021,10.513670,...,9.258468,9.063374,8.588468,9.649255,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,10.014838,10.013358,10.164495,10.009226,10.024938,9.821374,9.865287,10.252872,10.066131,10.501209,...,8.508420,8.417641,7.922295,8.536418,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,9.874201,9.613174,9.428842,9.855727,9.184982,9.542309,9.898331,10.138428,10.234679,10.439282,...,7.548015,8.238778,8.112399,8.383661,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,10.383832,10.089753,10.019128,10.162333,10.204617,10.145849,10.132951,9.966949,9.962975,10.546733,...,8.496480,7.431211,8.005023,8.612472,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
RZUH02_20220906_043000.WAV,10.106177,10.247113,10.203444,10.545758,10.467957,10.384063,10.207061,10.170845,10.103601,10.529855,...,8.034658,8.641006,8.365255,9.214890,RZUH02_20220906_043000.WAV,RZUH02,06,04,night,pasture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RZUF12_20220912_091500.WAV,10.079299,9.814939,9.708104,9.954927,9.650657,9.480253,9.534870,9.609855,9.787546,10.499600,...,8.690336,8.706616,8.728745,8.709227,RZUF12_20220912_091500.WAV,RZUF12,12,09,day,pasture
RZUF12_20220912_091500.WAV,9.865605,9.606849,9.494812,9.605864,9.598310,9.650576,9.648650,9.550693,9.592554,10.504624,...,8.722764,8.727214,8.733954,8.718722,RZUF12_20220912_091500.WAV,RZUF12,12,09,day,pasture
RZUF12_20220912_091500.WAV,9.500553,9.652675,9.749570,9.509336,9.613685,9.385221,9.460393,9.390950,9.568528,10.377753,...,8.724422,8.700010,8.728350,8.670266,RZUF12_20220912_091500.WAV,RZUF12,12,09,day,pasture


In [37]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

df_day={}
accuracies = []
f1_scores = []
recalls = []
for i in ["03","04","05","06","07","08","09","10","11","12","13"]:
    df_day = df_ae[df_ae['day'].isin([i])]
    X = np.asarray(df_day.loc[:,0:5183])
    y = np.asarray(df_day.loc[:,"cover"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
    clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
    clf_rf.fit(X_train, y_train)
    y_pred_rf = clf_rf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred_rf)
    f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
    recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
    print("Accuracy:", accuracy)
    print("f1:", f1_score)
    print("recall", recall)

    accuracies.append(accuracy)
    f1_scores.append(f1_score)
    recalls.append(recall)

Accuracy: 0.9141370338248048
f1: 0.8673828707471878
recall 0.8348418621946183
Accuracy: 0.8830420969023034
f1: 0.8339522822827549
recall 0.8052500720676252
Accuracy: 0.8700536124585142
f1: 0.8369116336160406
recall 0.812328480861756
Accuracy: 0.8735632183908046
f1: 0.8391752910085758
recall 0.8188907603451603
Accuracy: 0.854147571035747
f1: 0.8103965582593328
recall 0.7846954031955398
Accuracy: 0.8146658014276443
f1: 0.7571842263779068
recall 0.7273010706429103
Accuracy: 0.8739963952154678
f1: 0.8412798848228865
recall 0.8220469771343012
Accuracy: 0.8706231454005935
f1: 0.8360641698497235
recall 0.8125289577426481
Accuracy: 0.8730871823669802
f1: 0.8381251747789743
recall 0.814331955370343
Accuracy: 0.8839205058717254
f1: 0.8492345520491629
recall 0.8202955984493915
Accuracy: 0.9515287099179717
f1: 0.9194428517736788
recall 0.8920508560337059


In [30]:
np.asarray(X).shape

(6705, 5184)

### Autoencoders Features and Labels using independent segments

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_ae, labels_ae, test_size=0.2,random_state=0)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, y_train)

y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred_rf)
f1_score = metrics.f1_score(y_test, y_pred_rf, average="macro")
recall = metrics.recall_score(y_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

### Autoencoders Features and Labels using 5 segments of the audio

In [ ]:
X_batch = np.reshape(X_ae, (X_ae.shape[0]//5,5,X_ae.shape[1]))
y_batch = np.reshape(y_ae, (y_ae.shape[0]//5, 5))

In [ ]:
y_3 == y_path[:,0]

In [ ]:
len(labels_ai), y_path[0].shape

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_batch, y_path, test_size=0.2,random_state=0)

X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1], X_test.shape[2])
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = y_train.reshape(y_train.shape[0]*y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1])

labels_train = []
for i in range(len(y_train)):
    labels_train.append(audios.loc[y_train[i], "cover"])
labels_test = []
for i in range(len(y_test)):
    labels_test.append(audios.loc[y_test[i], "cover"])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf_rf = RandomForestClassifier(max_depth=32, random_state=0, n_jobs=-1)
clf_rf.fit(X_train, labels_train)
y_pred_rf = clf_rf.predict(X_test)
accuracy = metrics.accuracy_score(labels_test, y_pred_rf)
f1_score = metrics.f1_score(labels_test, y_pred_rf, average="macro")
recall = metrics.recall_score(labels_test, y_pred_rf, average="macro")
print("Accuracy:", accuracy)
print("f1:", f1_score)
print("recall", recall)

# Autoencoders Features and Labels using voting labels

In [ ]:
y_pred_rf2 = np.asarray(y_pred_rf)
y_pred_rf2 = np.reshape(y_pred_rf2,(y_pred_rf2.shape[0]//5,5))
y_test2 = np.asarray(labels_test)
y_test2 = np.reshape(y_test2,(y_test2.shape[0]//5,5))

def most_frequent(List):
    return max(set(List), key = List.count)

labels_test2 = []
labels_pred = []
for i in range(len(y_pred_rf2)):
    labels_pred.append(most_frequent(list(y_pred_rf2[i])))
    labels_test2.append(most_frequent(list(y_test2[i])))
accuracy_ae = metrics.accuracy_score(labels_test2, labels_pred)
f1_score_ae = metrics.f1_score(labels_test2, labels_pred, average="macro")
recall_ae = metrics.recall_score(labels_test2, labels_pred, average="macro")

print("Accuracy:", accuracy_ae)
print("f1:", f1_score_ae)
print("recall", recall_ae)